In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable
from chainer import optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [3]:
# Set data
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
Y = iris.target.astype(np.float32)
N = Y.size
Y2 = np.zeros(3 * N).reshape(N,3).astype(np.float32)
for i in range(N):
    Y2[i,np.int(Y[i])] = 1.0

index = np.arange(N)
xtrain = X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest = X[index[index % 2 == 0],:]
yans = Y[index[index % 2 == 0]]

In [3]:
# Define model

class IrisRogi(Chain):
    def __init__(self):
        super(IrisRogi, self).__init__(
            l1=L.Linear(4,3),
        )

    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x), y)

    def fwd(self,x):
        return F.softmax(self.l1(x))

# Initialize model

model = IrisRogi()
optimizer = optimizers.Adam()
optimizer.setup(model)

In [4]:
# Learn

n = 75
bs = 25
for j in range(5000):
    sffindx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = Variable(xtrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        y = Variable(ytrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        model.zerograds()
        loss = model(x,y)
        loss.backward()
        optimizer.update()

In [5]:
# Test

xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)

ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    print( ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print (ok, "/", nrow, " = ", (ok * 1.0)/nrow)

[  9.98923481e-01   1.07652182e-03   1.62120678e-10] 0
[  9.97842550e-01   2.15742411e-03   8.67760530e-10] 0
[  9.99105513e-01   8.94484809e-04   1.28208263e-10] 0
[  9.97292221e-01   2.70777266e-03   1.74718240e-09] 0
[  9.92561877e-01   7.43814977e-03   1.20575541e-08] 0
[  9.99266207e-01   7.33762630e-04   5.98604569e-11] 0
[  9.96840239e-01   3.15974583e-03   1.40284140e-09] 0
[  9.99891996e-01   1.07970664e-04   1.05179808e-12] 0
[  9.99505162e-01   4.94780776e-04   3.65727275e-11] 0
[  9.98757005e-01   1.24296767e-03   1.40746970e-10] 0
[  9.97154593e-01   2.84544285e-03   7.70414343e-10] 0
[  9.99659538e-01   3.40523809e-04   2.95169444e-11] 0
[  9.92364526e-01   7.63550820e-03   9.55877688e-09] 0
[  9.94491339e-01   5.50867338e-03   5.18369569e-09] 0
[  9.98704433e-01   1.29556062e-03   2.04996423e-10] 0
[  9.93540585e-01   6.45942148e-03   6.43222497e-09] 0
[  9.99784529e-01   2.15459513e-04   6.70674670e-12] 0
[  9.96863604e-01   3.13639618e-03   1.28069810e-09] 0
[  9.99384

In [6]:
import matplotlib.pyplot as plt
x = Variable(xtrain)
yt = F.sigmoid(model.l1(x))
ans = yt.data
ansx1 = ans[0:50, 0]
ansy1 = ans[0:50, 1]
ansx2 = ans[50:100, 0]
ansy2 = ans[50:100, 1]
ansx3 = ans[100:150, 0]
ansy3 = ans[100:150, 1]
plt.scatter(ansx1, ansy1, marker="^")
plt.scatter(ansx2, ansy2, marker="o")
plt.scatter(ansx3, ansy3, marker="+")
plt.show()

